<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Two-dimensional-non-convex-problem" data-toc-modified-id="Two-dimensional-non-convex-problem-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Two dimensional non-convex problem</a></span><ul class="toc-item"><li><span><a href="#Gradient-algorithm" data-toc-modified-id="Gradient-algorithm-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Gradient algorithm</a></span><ul class="toc-item"><li><span><a href="#Fixed-step-size-gradient-algorithm" data-toc-modified-id="Fixed-step-size-gradient-algorithm-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Fixed step size gradient algorithm</a></span></li><li><span><a href="#Receeding-step-size" data-toc-modified-id="Receeding-step-size-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Receeding step size</a></span></li></ul></li><li><span><a href="#Conjugate-gradient" data-toc-modified-id="Conjugate-gradient-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Conjugate gradient</a></span></li><li><span><a href="#Newton's-method" data-toc-modified-id="Newton's-method-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Newton's method</a></span></li><li><span><a href="#Quasi-Newton-Method" data-toc-modified-id="Quasi-Newton-Method-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Quasi-Newton Method</a></span></li><li><span><a href="#Plotting-multiple-trajectories" data-toc-modified-id="Plotting-multiple-trajectories-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Plotting multiple trajectories</a></span></li></ul></li><li><span><a href="#Quadratic-function-and-conditionning" data-toc-modified-id="Quadratic-function-and-conditionning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Quadratic function and conditionning</a></span></li><li><span><a href="#Non-convex-function-in-higher-dimension" data-toc-modified-id="Non-convex-function-in-higher-dimension-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Non-convex function in higher dimension</a></span></li></ul></div>

# Testing multiple optimization method

We start by adding some libraries.

In [ ]:
# uncomment and run once to install
 using Pkg
#Pkg.add("Plots")
#Pkg.add("LinearAlgebra")
#Pkg.add("Optim")


In [ ]:
Pkg.add("PlotlyJS")
Pkg.add("ORCA")# if you want to try plotlyjs()

In [ ]:
# run to make the library available
using Plots, LinearAlgebra, Optim 
plotlyjs() #Plots has multiple plotting backend. plotlyjs() has more interactivity in notebook but sometimes buggy
#pyplot()

## Two dimensional non-convex problem

We define the Rosenbrock (a.k.a banana) function $f(x,y) = (1-x)^2 + 5(y-x^2)^2$. 

It is a function well known to be difficult to optimize.

In [ ]:
# Run to define the objective function, its gradient hessian, and plot its level set
# might take some time to run
f = x -> (1-x[1])^2 + 5*(x[2] - x[1]^2)^2
df = x -> [2*(10*x[1]^3-10*x[1]*x[2]+x[1]-1), 10*(x[2]-x[1]^2)]
hf = x -> [60x[1]^2-20x[2]+2 -20x[1]; -20x[1] 10]

#
x0 = Float64[-0.5, 2] #Initial point
ε = 1e-6              #tolerance for the stopping test, here the gradient
IT_MAX = 5000         #max number of iterations, always a good idea to have this as fail-safe

xdomain = -2:0.01:2
ydomain = -2:0.01:2

lev_line = contour(xdomain,ydomain,(x,y)->f([x,y]), levels = 200);
lev_line = plot!(lev_line,[1],[1],seriestype = :scatter, color = :red,label="solution");
lev_line

The following function plot the trajectory of an algorithm on this example.

In [ ]:
# The last argument is used to plot on top of an already computed plot. 
# By default it is the contour level set
function plot_trajectory(pts;color="blue",label=`AUTO`,shape=:cross,plt=lev_line)
    plt = deepcopy(plt)
    plot!(plt,[p[1] for p in pts],[p[2] for p in pts],color=color,label=label,shape=shape)
    return plt
end


The following function plots graph showing the convergence of the algorithm

In [ ]:
function plot_convergence(vals,grads,steps)
    p1 = plot(vals, title = "Objective value")
    p2 = plot(log10.(vals .- vals[end]),title = "log gap value")
    p3 = plot(log10.(norm.(grads)), title = "log gradient norm") 
    p4 = plot(steps, title = "Step-size")
    plot(p1, p2, p3,p4, layout = (2, 2), legend = false)
end

### Gradient algorithm

We start with the most classical descent direction algorithm.

#### Fixed step size gradient algorithm


<div class="alert alert-success"> <b>Question 1)</b> 
    
Numerically check that the algorithm does not converge for large step-size. Plot the corresponding trajectory.

Find the best fixed step-size. What do you observe ?
</div>


NB: I got 845 iterations.

<div class="alert alert-info" role="alert"><b>Question 1)</b>
Answer
</div>

In [ ]:
t0 = 0.02             #default step size



function gradient_fixed_step(x0,f,df;t0=t0)
    x = x0
    
    pts = [x]
    vals = [f(x)]
    grads = [df(x)]
    steps = [t0]

    stop_test = false
    it = 0 #iteration number

    while !stop_test
        # update
        g = df(x)
        d = -g             # descent direction
        t = t0             # step size
        x += t*d           # update

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        push!(steps,t0)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("gradient algorithm with fixed step t=",t0, " ended in ", it, " iterations and ", it," oracle call")
    return pts, vals, grads, steps
   
end

pts_gf, vals, grads, steps = gradient_fixed_step(x0,f,df,t0=0.02);
plt = plot_trajectory(pts_gf,label="gradient") # plotting the trajectory

In [ ]:
    plot_convergence(vals,grads,steps)

#### Receeding step size

We are now going to use a receeding step size.



<div class="alert alert-success"> <b>Question 2)</b> 
    
Complete the receeding_step code.

Play with the parameters of the gradient with receeding step to get the smallest number of oracle calls.
</div>
NB : I got 267 calls.

<div class="alert alert-info" role="alert"><b>Question 2)</b>
Answer
</div>

In [ ]:
function receeding_step(f,g,x,d;reducing_factor=.5,slope_coefficient=0.5,it_max = 10, verbose = true, t0=1)
    f0 = f(x)
    t = t0
    it = 1
    t_traj = [t]
    
    ### TO COMPLETE 
    while ( ... ) && (it <= it_max)
        t = ### TO COMPLETE
        push!(t_traj,t)
        it += 1
    end
    
    if verbose
        f(x+t*d) >= f0 && println("non reducing step ", "it = ", it)
        it==it_max + 1 && println("maximum number of receeding iterations attained") 
    end
    return t_traj
end

In [ ]:
function gradient_receeding_step(x0,f,df;
        t0=t0, reducing_factor=.9,slope_coefficient=0.5,it_max = 10, verbose = false)
    x = x0
    
    pts = [x]
    pts_full = [x]
    vals = [f(x)]
    grads = [df(x)]
    steps = []

    stop_test = false
    it = 0 #iteration number
    calls = 0

    while !stop_test
         
        g = df(x)
        d = - g       # descent direction
        t_traj = receeding_step(f,g,x,d;
            reducing_factor=reducing_factor,slope_coefficient=slope_coefficient,
            it_max = it_max, verbose = verbose, t0=t0)# step size
        calls += length(t_traj)
        t = t_traj[end]
        for t in t_traj
            push!(pts_full, x+t*d)
        end
        x += t*d          # step

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        push!(steps,t)
        
        
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("gradient algorithm with receeding step size ended in ", it, " iterations and ", calls," oracle calls")

    return pts, pts_full, vals, grads, steps

end

pts_gr, pts_gr_full, vals, grads, steps = gradient_receeding_step(x0,f,df;verbose = false);

In [ ]:
plt = plot_trajectory(pts_gr_full,label="gradient receeding",shape = :diamond,color = :yellow)
plt = plot_trajectory(pts_gr,label="gradient receeding",plt=plt)


In [ ]:
plot_convergence(vals,grads,steps)

### Conjugate gradient

We implement a conjugate gradient method.

<div class="alert alert-success"> <b>Question 3)</b> 

Implement a conjugate gradient method with receeding line search. You can test Fletcher-Reeves and Polak-Ribière version.
What is the smallest oracle calls you can get ?
</div>
NB : I got 178


<div class="alert alert-info" role="alert"><b>Question 3)</b>
Answer
</div>

In [ ]:
function conjugate_gradient_receeding_step(x0,f,df;t0=0.02, reducing_factor=.9,slope_coefficient=0.5,it_max = 10, verbose = false)
    x = x0
    
    pts = [x]
    pts_full=[x]
    vals = [f(x)]
    grads = [df(x)]
    steps = []

    stop_test = false
    it = 0 #iteration number
    calls = 0
    
    #first iteration
    g = df(x)
    d = - g       # descent direction
    t_traj = receeding_step(f,g,x,d;
            reducing_factor=reducing_factor,slope_coefficient=slope_coefficient,
            it_max = it_max, verbose = verbose, t0=t0)# step size
    calls += length(t_traj)
    t = t_traj[end]
        for t in t_traj
            push!(pts_full, x+t*d)
        end
    x += t*d          # step

    # keeping track of results
    push!(pts,x)
    push!(vals,f(x))
    push!(grads,g)
    push!(steps,t)
    
    
    while !stop_test
        ## 
        ## TO COMPLETE
        ## d =   # descent direction
        t_traj = receeding_step(f,g,x,d;
            reducing_factor=reducing_factor,slope_coefficient=slope_coefficient,
            it_max = it_max, verbose = verbose, t0=t0)# step size
        calls += length(t_traj)
        t = t_traj[end]
        for t in t_traj
            push!(pts_full, x+t*d)
        end
        x += t*d          # step

    # keeping track of results
    push!(pts,x)
    push!(vals,f(x))
    push!(grads,g)
    push!(steps,t)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("conjugate gradient algorithm with receeding step size ended in ", it, " iterations and ", calls," oracle calls")
    return pts,pts_full, vals, grads, steps
   
end

pts_cg,pts_full_cg, vals, grads,steps = conjugate_gradient_receeding_step(x0,f,df;t0=0.02, reducing_factor=.9,slope_coefficient=0.5,it_max = 10, verbose = false));
plt = plot_trajectory(pts_full_cg,label="conjugate gradient",shape = :diamond,color = :yellow)
plt = plot_trajectory(pts_cg,label="conjugate gradient",plt=plt)

In [ ]:
plot_convergence(vals,grads,steps)

### Newton's method

<div class="alert alert-success"> <b>Question 4)</b> 

Implement the Newton method with reducing step. 
Test-it and plot the trajectory. What do you observe ? 

Identify the problem and suggest an easy fix.
</div>


NB : I get 20 oracles calls.


NBB : Funnily enough, on this particular example, doing things poorly yield the solution quicker, in 7 calls. Do not generalize this result...

<div class="alert alert-info" role="alert"><b>Question 4)</b>
Answer
</div>

In [ ]:
x0 = Float64[-0.5, 2]

function Newton(x0,f,df, hf;
        reducing_factor=.9,slope_coefficient=0.5,it_max = 10, verbose = false)
    x = x0
    t0 = 1.0
    
    pts = [x]
    pts_full = [x]
    vals = [f(x)]
    grads = [df(x)]
    steps=[]
    
    
    stop_test = false
    it = 0 #iteration number
    calls = 0
    
    while !stop_test
        ### TO COMPLETE
        g = df(x)         # gradient
        H = hf(x)         # hessian
        d =       # descent direction 
        
       
        ### END COMPLETION
        t_traj = receeding_step(f,g,x,d;
            reducing_factor=reducing_factor,slope_coefficient=slope_coefficient,
            it_max = it_max, verbose = verbose, t0=t0)# step size
        calls += length(t_traj)
        t = t_traj[end]
        for t in t_traj
            push!(pts_full, x+t*d)
        end
        x += t*d          # step

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        push!(steps,t)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("Newton algorithm ended in ", it, " iterations and ", calls, " calls")
    return pts, pts_full, vals, grads, steps
   
end

pts_n,pts_n_full, vals,grads,steps = Newton(x0,f,df, hf; reducing_factor=.9,slope_coefficient=0.5,it_max = 10, verbose = false)


plt = plot_trajectory(pts_n_full,label="Newton",shape = :diamond,color = :yellow)
plt = plot_trajectory(pts_n,label="Newton",plt=plt)

In [ ]:
plot_convergence(vals,grads,steps)

### Quasi-Newton Method 

<div class="alert alert-success"> <b>Question 5)</b> 
Implement now the BFGS method, and adjust to get smallest oracle calls.
</div>

NB : I got 24 oracle calls

<div class="alert alert-info" role="alert"><b>Question 5)</b>
Answer
</div>

In [ ]:
function BFGS(x0,f,df;
        reducing_factor=.7,slope_coefficient=0.5,it_max = 10, verbose = false)
    x = x0
    
    pts = [x]
    pts_full = [x]
    vals = [f(x)]
    grads = [df(x)]
    steps = []
    
    stop_test = false
    it = 0 #iteration number
    calls = 0
    
    g_old = df(x)
    x_old = x0
    W = I
    
    while !stop_test
        
        g = df(x)         # gradient
        
        ## Computing W
        
        
        d = - W*g       # descent direction 
        
        
        if g'*d >= 0 
            println("non-decreasing direction at it ", it,", using gradient step instead")
            d = - g
        end
        t_traj = receeding_step(f,g,x,d;
            reducing_factor=reducing_factor,slope_coefficient=slope_coefficient,
            it_max = it_max, verbose = verbose, t0=1.0)# step size
        calls += length(t_traj)
        t = t_traj[end]
        for t in t_traj
            push!(pts_full, x+t*d)
        end
        x += t*d          # step

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        push!(steps,t)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("BFGS algorithm ended in ", it, " iterations and ", calls, " calls")
    return pts,pts_full, vals, grads, steps
   
end

pts_bfgs,pts_bfgs_full, vals,grads, steps = BFGS(x0,f,df; 
    reducing_factor=.7,slope_coefficient=0.5,it_max = 10, verbose = false)
    
plt = plot_trajectory(pts_bfgs_full,label="BFGS",shape = :diamond,color = :yellow)
plt = plot_trajectory(pts_bfgs,label="BFGS",plt=plt)

In [ ]:
plot_convergence(vals,grads,steps)

### Plotting multiple trajectories

To compare the algorithm we can plot multiple trajectories on the same graph.

In [ ]:
plt=plot_trajectory(pts_gf,label="gradient fixed",color="yellow")
plt=plot_trajectory(pts_gr,label="gradient adaptative",color="orange",plt=plt)
plt=plot_trajectory(pts_cg,label="conjugate gradient",color="blue",plt=plt)
plt=plot_trajectory(pts_n,label="Newton",color="red",plt=plt)
plt=plot_trajectory(pts_bfgs,label="BFGS",color="green",plt=plt)

## Quadratic function and conditionning

We are now going to test the effect of the condition number on the algorithm


<div class="alert alert-success"> <b>Question 6)</b> 

Test the algorithms on quadratic function. 
Comment on the dependence in the dimension and conditionning.
</div>


<div class="alert alert-info" role="alert"><b>Question 6)</b>
Answer
</div>

In [ ]:
n = 500        # dimension
κ = 100        # condition number

# generating a random symetric matrix of condition κ
X = rand(n,n) 
Q = X'X
U,S,V = svd(Q) 
γ = (κ-1)/(S[1]-S[end] ) # scaling factor
S = [(S[i]-S[end])*γ + 1 for i in 1:n]
Q = U*diagm(S)*V'

# defining the oracle
f_Q(x) = 0.5*x'*Q*x
df_Q(x) = Q*x
hf_Q(x) = Q

# Initial point
x0 = ones(n); # the optimal solution is obviously 0, so the initial point should be something else


In [ ]:
t0 = 0.5
gradient_fixed_step(x0,f_Q,df_Q;t0=t0);
gradient_receeding_step(x0,f_Q,df_Q;t0=t0);
conjugate_gradient_receeding_step(x0,f_Q,df_Q;t0=t0);
Newton(x0,f_Q,df_Q,hf_Q);
BFGS(x0,f_Q,df_Q);

<div class="alert alert-warning"> <b>Question 7) [Optional]</b> 
In class we said that the conjugate gradient algorithm should converge in at most $n$ iterations. Do we observe this here ? Why ? 

Write a conjugate gradient method that would satisfy this convergence.
</div>

<div class="alert alert-info" role="alert"><b>Question 7)</b>
Answer
</div>

<div class="alert alert-warning"> <b>Question 8) [Optional]</b> 

Write a conjugate gradient method for solving a random linear system $Ax=b$. 

Test it with random matrices.
</div>

<div class="alert alert-info" role="alert"><b>Question 8)</b>
Answer
</div>

## Non-convex function in higher dimension

We will now test our algorithms on the Rosenbrock function in higher dimension.

$$ f_\kappa(x) = \sum_{i=1}^{n-1} \kappa (x_{i+1} - x_i^2)^2 + (x_i -1)^2 $$

for $\kappa \geq 0$.

<div class="alert alert-warning"> <b>Question 9) [Optional]</b> 

Find, for any $\kappa \geq 0$ the global minimum of $f_\kappa$. Is it unique ?

Compute the gradient and the hessian of $f_{\kappa}$.
</div>

<div class="alert alert-info" role="alert"><b>Question 9)</b>
Answer
</div>

<div class="alert alert-warning"> <b>Question 10) [Optional]</b> 

Test the algorithms for various dimensions, starting points and $\kappa$.
Comment your numerical observations.

NB : The standard Rosenbrock function use $\kappa = 100$ and is known to be difficult to solve. 
</div>

<div class="alert alert-info" role="alert"><b>Question 10)</b>
Answer
</div>